# PyData London 2017

# Foreign Function Interface Generator: Generating Python bindings from C++

### Jonathan B Coe
### jbcoe@ffig.org

## https://github.com/ffig/ffig

We want to run C++ code in Python without doing any extra work.

## Gathering Input

Write a C++ class out to a file in the current working directory

In [1]:
outputfile = "Shape.h"

In [2]:
%%file $outputfile

#include "ffig/attributes.h"
#include <stdexcept>
#include <string>

struct FFIG_EXPORT Shape
{
  virtual ~Shape() = default;
  virtual double area() const = 0;
  virtual double perimeter() const = 0;
  virtual const char* name() const = 0;
};

static const double pi = 3.14159;

class Circle : public Shape
{
  const double radius_;

public:
  double area() const override
  {
    return pi * radius_ * radius_;
  }

  double perimeter() const override
  {
    return 2 * pi * radius_;
  }

  const char* name() const override
  {
    return "Circle";
  }

  Circle(double radius) : radius_(radius)
  {
    if ( radius < 0 ) 
    { 
      std::string s = "Circle radius \"" + std::to_string(radius_) + "\" must be non-negative.";
      throw std::runtime_error(s);
    }
  }
};

Overwriting Shape.h


Compile our header to check it's valid C++

In [3]:
%%sh
clang++ -x c++ -fsyntax-only -std=c++14 -I../ffig/include Shape.h 

Read the code using libclang

In [4]:
import sys
sys.path.insert(0,'../ffig')
sys.path.insert(0,'..')

In [5]:
import ffig.clang.cindex

index = ffig.clang.cindex.Index.create()
translation_unit = index.parse(outputfile, ['-x', 'c++', '-std=c++14', '-I../ffig/include'])

In [6]:
import asciitree

def node_children(node):
    return (c for c in node.get_children() if c.location.file.name == outputfile)

print(asciitree.draw_tree(translation_unit.cursor,
  lambda n: [c for c in node_children(n)],
  lambda n: "%s (%s)" % (n.spelling or n.displayname, str(n.kind).split(".")[1])))


Shape.h (TRANSLATION_UNIT)
  +--Shape (STRUCT_DECL)
  |  +--GENERATE_C_API (ANNOTATE_ATTR)
  |  +--~Shape (DESTRUCTOR)
  |  |  +-- (COMPOUND_STMT)
  |  +--area (CXX_METHOD)
  |  +--perimeter (CXX_METHOD)
  |  +--name (CXX_METHOD)
  +--pi (VAR_DECL)
  |  +-- (FLOATING_LITERAL)
  +--Circle (CLASS_DECL)
     +--struct Shape (CXX_BASE_SPECIFIER)
     |  +--struct Shape (TYPE_REF)
     +--radius_ (FIELD_DECL)
     +-- (CXX_ACCESS_SPEC_DECL)
     +--area (CXX_METHOD)
     |  +-- (CXX_OVERRIDE_ATTR)
     |  +-- (COMPOUND_STMT)
     |     +-- (RETURN_STMT)
     |        +-- (BINARY_OPERATOR)
     |           +-- (BINARY_OPERATOR)
     |           |  +--pi (UNEXPOSED_EXPR)
     |           |  |  +--pi (DECL_REF_EXPR)
     |           |  +--radius_ (UNEXPOSED_EXPR)
     |           |     +--radius_ (MEMBER_REF_EXPR)
     |           +--radius_ (UNEXPOSED_EXPR)
     |              +--radius_ (MEMBER_REF_EXPR)
     +--perimeter (CXX_METHOD)
     |  +-- (CXX_OVERRIDE_ATTR)
     |  +-- (COMPOUND_STM

Turn the AST into some easy to manipulate Python classes

In [7]:
import ffig.cppmodel
import ffig.clang.cindex

In [8]:
model = ffig.cppmodel.Model(translation_unit=translation_unit, force_noexcept=False)

In [9]:
[f.name for f in model.functions][-5:]

['to_wstring', 'operator""s', 'operator""s', 'operator""s', 'operator""s']

In [10]:
[c.name for c in model.classes][-5:]

['__basic_string_common', 'basic_string', 'basic_string', 'Shape', 'Circle']

In [11]:
shape_class = [c for c in model.classes if c.name=='Shape'][0]

In [12]:
["{}::{}".format(shape_class.name,m.name) for m in shape_class.methods]

['Shape::area', 'Shape::perimeter', 'Shape::name']

## Code Generation

We now have some input to use in a code generator.

Look at the templates the generator uses

In [13]:
%cat ../ffig/templates/json.tmpl

{
  "name" : "{{class.name}}"{% if class.methods %},
  "methods" : [{% for method in class.methods %}
    {
      "name" : "{{method.name}}",
      "return_type" : "{{method.return_type}}"
    }{% if not loop.last %},{% endif %}{% endfor %}
  ]{% endif %}
}


Run the code generator

In [14]:
%%sh
cd ../
python -m ffig -b _c.h.tmpl _c.cpp.tmpl json.tmpl python -m Shape -i demos/Shape.h -o demos

See what it created

In [15]:
%ls

CMakeCache.txt              Shape.h
CMakeFiles/                 Shape.json
CMakeLists.txt              Shape_c.cpp
LLVM-Cauldron.ipynb.broken  Shape_c.h
Makefile                    cmake_install.cmake
PyDataLondon-2017.ipynb     libShape_c.dylib*
Shape/


In [16]:
%cat Shape.json

{
  "name" : "Shape",
  "methods" : [
    {
      "name" : "area",
      "return_type" : "double"
    },
    {
      "name" : "perimeter",
      "return_type" : "double"
    },
    {
      "name" : "name",
      "return_type" : "const char *"
    }
  ]
}

Build some bindings with the generated code.

In [17]:
%%file CMakeLists.txt

cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)

include_directories(../ffig/include)

add_library(Shape_c SHARED Shape_c.cpp)

Overwriting CMakeLists.txt


In [18]:
%%sh
cmake . 
cmake --build .

-- Configuring done
-- Generating done
-- Build files have been written to: /Users/jon/DEV/FFIG/demos
Scanning dependencies of target Shape_c
[ 50%] Building CXX object CMakeFiles/Shape_c.dir/Shape_c.cpp.o
[100%] Linking CXX shared library libShape_c.dylib
[100%] Built target Shape_c


In [19]:
%%sh
nm -U libShape_c.dylib | c++filt

0000000000003cf0 short GCC_except_table10
0000000000003d7c short GCC_except_table15
0000000000003ddc short GCC_except_table26
0000000000003bd0 short GCC_except_table6
0000000000003c30 short GCC_except_table8
0000000000003c90 short GCC_except_table9
00000000000025e0 T _Shape_Circle_create
00000000000022d0 T _Shape_Shape_area
0000000000002270 T _Shape_Shape_dispose
00000000000024e0 T _Shape_Shape_name
00000000000023e0 T _Shape_Shape_perimeter
0000000000002020 T _Shape_clear_error
00000000000021a0 T _Shape_error
0000000000004108 short Shape_error_
0000000000004248 short __ZL12Shape_error_$tlv$init
0000000000002f50 unsigned short Shape::Shape()
0000000000003050 unsigned short Shape::~Shape()
0000000000003030 unsigned short Shape::~Shape()
0000000000002f70 unsigned short Shape::~Shape()
0000000000002b80 unsigned short Circle::Circle(double)
0000000000002bb0 unsigned short Circle::Circle(double)
0000000000002fa0 unsigned short Circle::~Circle()
0000000000002f80 unsigned short Circle::~Circle

In [20]:
cat Shape/_py3.py

# This code was generated by FFIG <http://ffig.org>.
# Manual edits will be lost.

import os
from ctypes import *
c_object_p = POINTER(c_void_p)

class c_interop_string(c_char_p):

  def __init__(self, p=None):
    if p is None:
      p = ""
    if isinstance(p, str):
      p = p.encode("utf8")
    super(c_char_p, self).__init__(p)

  def __str__(self):
    return self.value

  @property
  def value(self):
    if super(c_char_p, self).value is None:
      return None
    return super(c_char_p, self).value.decode("utf8")

  @classmethod
  def from_param(cls, param):
    if isinstance(param, str):
      return cls(param)
    if isinstance(param, bytes):
      return cls(param)
    raise TypeError("Cannot convert '{}' to '{}'".format(type(param).__name__, cls.__name__))

  @staticmethod
  def to_python_string(x, *args):
    return x.value


class Shape_error(Exception):
    def __init__(self):
        self.value = conf.lib.Shape_error()
        conf.lib.Shape_clear_error()

    def __str_

In [21]:
%%python2
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

A Circle with radius 8 has area 201.06176


In [22]:
%%python3
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

A Circle with radius 8 has area 201.06176


In [23]:
%%script pypy
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

A Circle with radius 8 has area 201.06176


In [24]:
%%script /opt/intel/intelpython35/bin/python
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

A Circle with radius 8 has area 201.06176


In [25]:
from Shape import *

In [26]:
try:
    c = Circle(-8)
except Exception as e:
    print(e)

Circle radius "-8.000000" must be non-negative.


## FFIG needs you!

FFIG is MIT-licensed and hosted on GitHub.

Contributions, issues and feedback are very welcome.